In [36]:
import numpy as np
import copy
import time

In [37]:
class Node:
    def __init__(self, game, value, player, parent, action):
        self.game = game
        self.value = value
        self.player = player
        self.parent = parent
        self.action = action
        self.expandableActions = list(game.getPossibleActions())
        self.childs = []
        self.S = 0
        self.N = 0

    def is_expandable(self):
        return len(self.expandableActions) > 0
    
    def has_children(self):
        return len(self.childs) > 0

    def expand(self):
        if len(self.expandableActions) == 0:
            gamewon = self.game.checkVictory(self.action, -self.value)
            res = -self.value if gamewon else 0
            node = self
        else:
            action = np.random.choice(self.expandableActions)
            self.expandableActions.remove(action)

            game = copy.deepcopy(self.game)
            game.updateBoard(action, self.value)
            node = Node(game, self.value * -1, self.player, self, action)
            self.childs.append(node)
            
            gamewon = game.checkVictory(action, self.value)
            res = node.rollout() if gamewon == False else self.value
            res = res * self.player
        node.backpropogate(res)

    def rollout(self):
        game = copy.deepcopy(self.game)
        while True:
            for turn in [self.value, -self.value]:
                if len(game.getPossibleActions()) > 0:
                    action = np.random.choice(game.getPossibleActions())
                    
                    game.updateBoard(action, turn)
                    gameWon = game.checkVictory(action, turn)
                    if gameWon:
                        return turn
                    turn *= -1
                else:
                    return 0
    
    def backpropogate(self, value):
        node = self
        while node != None:
            node.S += value
            node.N += 1
            node = node.parent

    def select_child(self, C):
        if len(self.childs) > 0:
            UCBs = [child.UCB(C) for child in self.childs]
            max_i = np.argmax(UCBs)
            return self.childs[max_i]
        else:
            return None

    def UCB(self, C):
        # if self.N > 0 and self.parent.N > 0:
        return (self.S / self.N) + C * (np.sqrt(np.log(self.parent.N)/self.N))
        # else:
        #     return np.inf
    
    def decide(self):
        scores = [child.S for child in self.childs] # Nesu tikras ar čia tiks score, bet skamba logiškai
        max_i = np.argmax(scores)
        return self.childs[max_i]

class MCTS:
    def __init__(self, C, thinking_time):
        self.C = C
        self.thinking_time = thinking_time

    def think(self, game, value):
        root = Node(copy.deepcopy(game), value, value, None, None)
        start_time = time.time()
        expanded = 0
        while time.time() - start_time < self.thinking_time:
            node = self.select(root)
            node.expand()
            expanded += 1
        
        print(f"expanded: {expanded}; time spent: {time.time() - start_time}")
        return root.decide().action

    def select(self, node):
        while node.is_expandable() == False and node.has_children():
            node = node.select_child(self.C)
        return node

In [38]:
from game import gamerules

In [39]:
class MCTSPlayer(gamerules.Player):
    def __init__(self, name, mcts):
        super().__init__(name)
        self.mcts = mcts
    
    def getAction(self, board, value):
        action = self.mcts.think(board, value)
        return action

    def newGame(self, new_opponent):
        pass

In [40]:
from utils import play_game, test_games
from classes.player import RNGPlayer

In [41]:
mcts = MCTS(0.8, 1)
p1 = MCTSPlayer("Custom", mcts)
p2 = RNGPlayer()

In [42]:
test_games(p1, p2, 100)

expanded: 95; time spent: 1.0043740272521973
expanded: 116; time spent: 1.0000498294830322
expanded: 120; time spent: 1.006897211074829
expanded: 122; time spent: 1.0094072818756104
expanded: 171; time spent: 1.0101261138916016
expanded: 207; time spent: 1.000474214553833
expanded: 259; time spent: 1.0061984062194824
expanded: 238; time spent: 1.0000584125518799
expanded: 235; time spent: 1.0066711902618408
expanded: 171; time spent: 1.0000150203704834
expanded: 207; time spent: 1.0012047290802002
expanded: 234; time spent: 1.0009465217590332
expanded: 281; time spent: 1.0023188591003418
expanded: 337; time spent: 1.0020148754119873
expanded: 377; time spent: 1.0002377033233643
expanded: 465; time spent: 1.0014729499816895
expanded: 762; time spent: 1.0000896453857422
expanded: 1033; time spent: 1.0066306591033936
expanded: 4066; time spent: 1.0000133514404297
expanded: 6812; time spent: 1.0001535415649414
expanded: 10640; time spent: 1.0008823871612549
expanded: 129; time spent: 1.003